In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
# Core packages
import pygmo as pg
import numpy as np
from math import pi
from dotmap import DotMap

# For creating mesh
import mesh_utility

# For computing the next state
import equations_of_motion

# For optimization using pygmo
from udp_initial_condition import udp_initial_condition

# For computing trajectory
import trajectory_tools

# For plotting
import plotting_utility

# For cProfile evaluation
import cProfile
import pstats

In [44]:
def setup_parameters():
    """Set up of required hyperparameters for the optimization scheme. 

    Returns:

        body_args (dotmap.DotMap): Parameters related to the celestial body:
            density (float): Body density of celestial body.
            mu (float): Gravitational parameter for celestial body.
            declination (float): Declination angle of spin axis.
            right_ascension (float): Right ascension angle of spin axis.
            spin_period (float): Rotational period around spin axis of the body.
            spin_velocity (float): Angular velocity of the body's rotation.
            spin_axis (np.ndarray): The axis around which the body rotates.

        integrator_args (dotmap.DotMap): Specific parameters related to the integrator:
            algorithm (int): Integer representing specific integrator algorithm.
            dense_output (bool): Dense output status of integrator.
            rtol (float): Relative error tolerance for integration.
            atol (float): Absolute error tolerance for integration.

        problem_args (dotmap.DotMap): Parameters related to the problem:
            start_time (float): Start time (in seconds) for the integration of trajectory.
            final_time (float): Final time (in seconds) for the integration of trajectory.
            initial_time_step (float): Size of initial time step (in seconds) for integration of trajectory.
            target_squared_altitude (float): Squared value of the satellite's orbital target altitude.
            radius_bounding_sphere (float): Radius of the bounding sphere representing risk zone for collisions with celestial body.
            event (int): Event configuration (0 = no event, 1 = collision with body detection)
            number_of_maneuvers (int): Number of possible maneuvers.
        
        lower_bounds (np.ndarray): Lower boundary values for the initial state vector.
        upper_bounds (np.ndarray): Lower boundary values for the initial state vector.
                
        population_size (int): Number of chromosomes to compare at each generation.
        number_of_generations (int): Number of generations for the genetic opimization.
    """
    # Define complete dotmap
    args = DotMap(
        body = DotMap(_dynamic=False),
        integrator = DotMap(_dynamic=False),
        problem = DotMap(_dynamic=False),
        mesh = DotMap(_dynamic=False),
        _dynamic=False)

    # Setup body parameters
    args.body.density = 533                  # https://sci.esa.int/web/rosetta/-/14615-comet-67p
    args.body.mu = 665.666                   # Gravitational parameter for 67P/C-G
    args.body.declination = 64               # [degrees] https://sci.esa.int/web/rosetta/-/14615-comet-67p
    args.body.right_ascension = 69           # [degrees] https://sci.esa.int/web/rosetta/-/14615-comet-67p
    args.body.spin_period = 12.06*3600       # [seconds] https://sci.esa.int/web/rosetta/-/14615-comet-67p
    args.body.spin_velocity = (2*pi)/args.body.spin_period
    args.body.spin_axis = equations_of_motion.setup_spin_axis(args)

    # Setup specific integrator parameters:
    args.integrator.algorithm = 3
    args.integrator.dense_output = True
    args.integrator.rtol = 1e-12
    args.integrator.atol = 1e-12

    # Setup problem parameters
    args.problem.start_time = 0                     # Starting time [s]
    args.problem.final_time = 20*3600               # Final time [s]
    args.problem.initial_time_step = 600            # Initial time step size for integration [s]
    args.problem.target_squared_altitude = 8000**2  # Target altitude squared [m]
    args.problem.activate_event = True              # Event configuration (0 = no events, 1 = collision with body detection)
    args.problem.number_of_maneuvers = 0            # Number of possible maneuvers for the spacecraft
    args.problem.selected_fitness_functions = [1, 2, 3, 4]
    args.problem.measurement_period = 10 # Period for when a measurement sphere is recognized and managed. Unit: [seconds]

    # Arguments concerning bounding spheres
    args.problem.radius_inner_bounding_sphere = 4000      # Radius of spherical risk-zone for collision with celestial body [m]
    args.problem.radius_outer_bounding_sphere = 10000
    args.problem.squared_volume_inner_bounding_sphere = (4/3)**2 * pi**2 * ((args.problem.radius_inner_bounding_sphere**2)**3)
    args.problem.squared_volume_outer_bounding_sphere = (4/3)**2 * pi**2 * ((args.problem.radius_outer_bounding_sphere**2)**3)
    args.problem.measurable_squared_volume = args.problem.squared_volume_outer_bounding_sphere - args.problem.squared_volume_inner_bounding_sphere
    
    # Defining the state variable and its boundaries (parameter space):
    #   state: [a, e, o, w, i, ea, tm, dvx, dvy, dvz]
    #
    # Declerations:
    #   a   : Semi-major axis
    #   e   : Eccentricity (e=[0,1]).
    #   o   : Right ascension of ascending node (o=[0,2*pi])
    #   w   : Argument of periapsis (w=[0,2*pi])
    #   i   : Inclination (i=[0,pi])
    #   ea  : Eccentric anomaly (ea=[0,2*pi])
    #   tm  : Time of impulsive maneuver ([seconds])
    #   dvx : Impulsive maneuver in x-axis
    #   dvy : Impulsive maneuver in y-axis
    #   dvz : Impulsive maneuver in z-axis
    #
    # NOTE: (Initial position and velocity are defined with osculating 
    #        orbital elements, i.e the first six parameters of the state vector)

    # Orbital elements
    a = [5000, 15000] 
    e = [0, 1]        
    o = [0, 2*pi]
    w = [0, 2*pi]     
    i = [0, pi]       
    ea = [0, 2*pi]

    # Impulsive Maneuver    
    tm = [(args.problem.start_time + 100), (args.problem.final_time - 100)]
    dvx = [-1, 1]
    dvy = [-1, 1]
    dvz = [-1, 1]
    
    # Generate boundary state vectors:
    lower_bounds = np.concatenate(([a[0], e[0], i[0], o[0], w[0], ea[0]], [tm[0], dvx[0], dvy[0], dvz[0]]*args.problem.number_of_maneuvers), axis=None)
    upper_bounds = np.concatenate(([a[1], e[1], i[1], o[1], w[1], ea[1]], [tm[1], dvx[1], dvy[1], dvz[1]]*args.problem.number_of_maneuvers), axis=None)

    # Optimization parameters
    population_size = 10
    number_of_generations = 32

    return args, lower_bounds, upper_bounds, population_size, number_of_generations

In [ ]:
# Main script:

def run():
    """
    Main function for optimizing the initial state for deterministic trajectories around a 
    small celestial body using a mesh.
    """
    
    print("Retrieving user defined parameters...")
    args, lower_bounds, upper_bounds, population_size, number_of_generations = setup_parameters()

    # Create the mesh
    print("Creating a mesh of the celestial body...")
    args.mesh.body, args.mesh.vertices, args.mesh.faces, args.mesh.largest_body_protuberant = mesh_utility.create_mesh()
    
    # Setup User-Defined Problem (UDP)
    print("Setting up the UDP...")
    udp = udp_initial_condition(args, lower_bounds, upper_bounds)
    prob = pg.problem(udp)

    # Setup optimization algorithm
    print("Setting up the optimization algorithm...")
    assert population_size >= 7

    # Create Differential Evolution object by passing the number of generations as input
    de_algo = pg.sade(gen = 1)

    # Create pygmo algorithm object
    algo = pg.algorithm(de_algo)

    # Create population
    pop = pg.population(prob = prob, size = population_size)

    # Initialize empty containers
    fitness_list = []

    print("Initializing optimization...")
    # Evolve population multiple times
    for i in range(number_of_generations):
        pop = algo.evolve(pop)
        fitness_list.append(pop.get_f()[pop.best_idx()])
        print("Generation: ", i+1, " "*10, "Best: ", fitness_list[len(fitness_list)-1])

    # Logs for output
    print("Champion fitness value: ", pop.champion_f) 
    print("Champion chromosome: ", pop.champion_x) 

    # Retrieving champion chromosome
    x = pop.champion_x

    # Re-compute optimized trajectory
    _, list_of_trajectory_objects, integration_intervals = trajectory_tools.compute_trajectory(x, args, equations_of_motion.compute_motion)
    trajectory_info = trajectory_tools.get_trajectory_info(args, list_of_trajectory_objects, integration_intervals)
    measurement_spheres_info = trajectory_tools.compute_measurement_spheres_info(args, list_of_trajectory_objects, integration_intervals)

    ######### Plotting results #########
    # Plot optimized trajectory around mesh
    view_angle = [15, -20]
    plotting_utility.plot_UDP(args, trajectory_info[0:3,:], True, True, True, view_angle, measurement_spheres_info)
    
    # 2-axis plot of the trajectory
    axis_1 = 0 #x-axis
    axis_2 = 1 #y-axis
    plotting_utility.two_axis_trajectory(trajectory_info, axis_1, axis_2)
    
    # Plot fitness over generations
    plotting_utility.fitness_over_generations(fitness_list, number_of_generations)    

def main():
    run()

if __name__ == "__main__":
    cProfile.run("main()", "output.dat")

    with open("output_time.txt", "w") as f:
        p = pstats.Stats("output.dat", stream=f)
        p.sort_stats("time").print_stats()
    
    with open("output_calls.txt", "w") as f:
        p = pstats.Stats("output.dat", stream=f)
        p.sort_stats("calls").print_stats()
        
        

In [51]:
l = np.array([2.51938511e+03,  2.32768836e+03, -9.42581658e+03])
n = np.linalg.norm(l)
print(n)

print(10000**2 - (l[0]**2 + l[1]**2 + l[2]**2))
print((10000 - n))

print(4000**2 - (l[0]**2 + l[1]**2 + l[2]**2))
print((4000 - n))


10030.526039723545
-611452.6335721016
-30.526039723545182
-84611452.6335721
-6030.526039723545


In [30]:
print(10000**2 - (2.57957739e+03)**2 - (1.99941309e+03)**2 - (-9.49729681e+03)**2)

-850518.9126987308


In [48]:
import numpy as np
import pykep as pk

# Setup args
args, lower_bounds, upper_bounds, population_size, number_of_generations = setup_parameters()
args.mesh.body, args.mesh.vertices, args.mesh.faces, args.mesh.largest_body_protuberant = mesh_utility.create_mesh()

# Setup initial state
x =  [9.72752753e+03, 3.64128116e-02, 1.28319173e+00, 1.69622584e+00, 2.18503270e+00, 2.66184179e+00]

args.integrator.t0 = 0
args.integrator.tf = 72000

r, v = pk.par2ic(E=x[0:6], mu=args.body.mu)
initial_state = np.array(r+v)


trajectory = trajectory_tools.integrate_system(equations_of_motion.compute_motion, initial_state, args)

In [49]:
print(trajectory.events)

(StateTuple(t=600.0, y=array([ 2.57957739e+03,  1.99941309e+03, -9.49729681e+03, -4.31789081e-02,
        2.46015774e-01,  4.21364208e-02]), event=<function point_is_outside_measurement_zone at 0x7fb19cb980d0>), StateTuple(t=1946.037352438132, y=array([ 2.51938511e+03,  2.32768836e+03, -9.42581658e+03, -4.56778324e-02,
        2.41665101e-01,  6.41296008e-02]), event=<function point_is_outside_measurement_zone at 0x7fb19cb980d0>), StateTuple(t=68740.40153712945, y=array([-9.28489296e+02, -1.71457964e+03,  3.48716579e+03, -2.22470013e-01,
        8.58105132e-01,  1.09821867e-01]), event=<function point_is_inside_risk_zone at 0x7fb19e781c10>), StateTuple(t=68932.68486783459, y=array([-9.71035028e+02, -1.54716092e+03,  3.50939331e+03, -2.19945446e-01,
        8.83127672e-01,  1.21599097e-01]), event=<function point_is_inside_risk_zone at 0x7fb19e781c10>), StateTuple(t=69131.9502993307, y=array([-1.01454007e+03, -1.36868467e+03,  3.53495005e+03, -2.16579693e-01,
        9.08014684e-01,  1.

In [ ]:

# Re-compute optimized trajectory
trajectory_info, squared_altitudes, collision_detected, measurement_spheres_info, measured_squared_volume = trajectory_tools.compute_trajectory(x, args, equations_of_motion.compute_motion)